In [1]:
from bert_serving.client import BertClient
import numpy as np
import pandas as pd

In [2]:
from tqdm import tqdm
from preprocess import read_articles_from_file_list, read_predictions_from_file, label2index
bc = BertClient(ip='10.2.0.111')

Using TensorFlow backend.


In [4]:
a = bc.encode(['First do it', 'then do it right', 'then do it better'])
a.shape

(3, 4096)

In [4]:
train_folder = "datasets/train-articles"  # check that the path to the datasets folder is correct,
dev_folder = "datasets/dev-articles"  # if not adjust these variables accordingly
test_folder = "datasets/test-articles"  # if not adjust these variables accordingly

train_labels_file = "datasets/train-task2-TC.labels"
dev_template_labels_file = "datasets/dev-task-TC-template.out"
# task_TC_output_file = "baseline-output-TC.txt"

test_template_labels_file = "datasets/test-task-TC-template.out"
# test_task_TC_output_file = "test-output-TC.txt"

In [5]:
def clean_text(text):
    # text = text.lower()
    # text = text.replace('\'', '')
    text = text.replace('‘', ' \' ')
    text = text.replace('’', ' \' ')
    text = text.replace('“', ' \" ')
    text = text.replace('”', ' \" ')
    text = text.replace('â', ' \' ')

    text = text.replace('"', ' " ')
    text = text.replace('\'', ' \' ')

    text = text.replace('—', ' - ')
    text = text.replace('–', ' - ')
    text = text.replace('…', '...')
    text = text.replace('  ', ' ')
    text = text.strip()
    return text

In [6]:
articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)
test_articles = read_articles_from_file_list(test_folder)
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)
test_article_ids, test_span_starts, test_span_ends, test_labels = read_predictions_from_file(test_template_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))


Loaded 6129 annotations from 357 articles


In [7]:
def compute_features(articles, ref_articles_id, span_starts, span_ends):
    # only one feature, the length of the span
    print(type(span_starts), len(span_starts))
    print(type(span_ends), len(span_ends))
    data = []
    article_spans = []
    for i, ref_id in tqdm(enumerate(ref_articles_id)):
        # print(articles[ref_id], span_starts[i], span_ends[i])
        article = articles[ref_id]
        article_span = clean_text(article[int(span_starts[i]):int(span_ends[i])])
        data.append([article_span])
        article_spans.append(article_span)
        
    return article_spans

In [8]:
articles = compute_features(articles, ref_articles_id, ref_span_starts, ref_span_ends)


6129it [00:00, 34163.16it/s]

<class 'list'> 6129
<class 'list'> 6129


In [9]:
dev_articles = compute_features(dev_articles, dev_article_ids, dev_span_starts, dev_span_ends)

1063it [00:00, 222105.47it/s]

<class 'list'> 1063
<class 'list'> 1063


In [10]:
test_articles = compute_features(test_articles, test_article_ids, test_span_starts, test_span_ends)

1790it [00:00, 230576.58it/s]

<class 'list'> 1790
<class 'list'> 1790


In [11]:
from keras.utils import to_categorical
from pprint import pprint
import pickle
from preprocess import label2index, index2label

In [12]:
GET_NEW_BERT_EMB = False

In [13]:
data_path = './bert_processed_data/'
if GET_NEW_BERT_EMB:
    articles_emb = bc.encode(articles)
    dev_articles_emb = bc.encode(dev_articles)
    test_articles_emb = bc.encode(test_articles)
    pprint(set(train_gold_labels))

    labels = [label2index[x] for x in train_gold_labels]
    labels = to_categorical(np.asarray(labels))

    # save train data
    pickle.dump(articles_emb, open(data_path + 'train_x.p', 'wb'))
    pickle.dump(labels, open(data_path + 'train_y.p', 'wb'))

    # save dev data
    pickle.dump(dev_articles_emb, open(data_path + 'dev_x.p', 'wb'))
    pickle.dump(dev_articles_emb, open(data_path + 'test_x.p', 'wb'))
    
else:
    articles_emb = pickle.load(open(data_path + 'train_x.p', 'rb'))
    labels = pickle.load(open(data_path + 'train_y.p', 'rb'))
    dev_articles_emb = pickle.load(open(data_path + 'dev_x.p', 'rb'))
    test_articles_emb = pickle.load(open(data_path + 'test_x.p', 'rb'))

/home/gangeshwark/anaconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=128" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_x, test_x, train_y, test_y = train_test_split(articles_emb,
                                                  labels,
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  stratify=labels)

In [16]:
from keras.layers import Conv2D, MaxPool2D, Bidirectional
from keras.layers import Embedding, Concatenate
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

def model_MLP(emb_size, out_size):
    model = Sequential()
    model.add(Dense(128, input_dim=emb_size, activation='relu'))
    model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(out_size, activation='softmax'))
    print(model.summary())
    return model

In [30]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from models import load_model


emb_size = articles_emb[0].shape[0]
print(emb_size)
out_size = 14

model = model_MLP(emb_size, out_size)

lr = 0.0001
bz = 256
epochs = 250

opt = Adam(lr=lr)
# opt = SGD(0.01)
# print(str(opt))
# exit()
min_loss_arg = -1
max_acc_arg = -1
model_name = 'bert_text_Adam_lr%s_bz%s' % (lr, bz)
model_path = 'models/%s' % (model_name)
checkpoint = ModelCheckpoint('%s.{epoch:02d}.hdf5' % (model_path), monitor='loss', verbose=1,
                             save_best_only=False, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
try:
    history = model.fit(train_x, train_y, validation_data=[test_x, test_y], epochs=epochs,
              batch_size=bz,
              shuffle=True, callbacks=[checkpoint])
    loss_hist = history.history['val_loss']
    acc_hist = history.history['val_acc']
    min_loss_arg = np.argmin(loss_hist)
    max_acc_arg = np.argmax(acc_hist)
except Exception as e:
    print(e)
    min_loss_arg = -1
    max_acc_arg = -1


4096
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 128)               524416    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 14)                1806      
Total params: 526,222
Trainable params: 526,222
Non-trainable params: 0
_________________________________________________________________
None
Train on 4903 samples, validate on 1226 samples
Epoch 1/250
4903/4903 [==============================] - 1s 216us/step - loss: 2.3650 - acc: 0.2554 - val_loss: 1.9362 - val_acc: 0.3646

Epoch 00001: saving model to models/bert_text_Adam_lr0.0001_bz256.01.hdf5
Epoch 2/250
4903/4903 [==============================] - 0s 31us/step - loss: 1.9396 - acc: 0.3781 - val_los

Epoch 36/250
4903/4903 [==============================] - 0s 31us/step - loss: 0.8512 - acc: 0.7293 - val_loss: 1.0754 - val_acc: 0.6615

Epoch 00036: saving model to models/bert_text_Adam_lr0.0001_bz256.36.hdf5
Epoch 37/250
4903/4903 [==============================] - 0s 27us/step - loss: 0.8319 - acc: 0.7361 - val_loss: 1.0720 - val_acc: 0.6607

Epoch 00037: saving model to models/bert_text_Adam_lr0.0001_bz256.37.hdf5
Epoch 38/250
4903/4903 [==============================] - 0s 30us/step - loss: 0.8241 - acc: 0.7404 - val_loss: 1.0734 - val_acc: 0.6648

Epoch 00038: saving model to models/bert_text_Adam_lr0.0001_bz256.38.hdf5
Epoch 39/250
4903/4903 [==============================] - 0s 32us/step - loss: 0.8159 - acc: 0.7373 - val_loss: 1.0717 - val_acc: 0.6599

Epoch 00039: saving model to models/bert_text_Adam_lr0.0001_bz256.39.hdf5
Epoch 40/250
4903/4903 [==============================] - 0s 30us/step - loss: 0.8087 - acc: 0.7485 - val_loss: 1.0707 - val_acc: 0.6631

Epoch 00040: s


Epoch 00074: saving model to models/bert_text_Adam_lr0.0001_bz256.74.hdf5
Epoch 75/250
4903/4903 [==============================] - 0s 29us/step - loss: 0.5598 - acc: 0.8311 - val_loss: 1.0651 - val_acc: 0.6648

Epoch 00075: saving model to models/bert_text_Adam_lr0.0001_bz256.75.hdf5
Epoch 76/250
4903/4903 [==============================] - 0s 29us/step - loss: 0.5603 - acc: 0.8354 - val_loss: 1.0641 - val_acc: 0.6664

Epoch 00076: saving model to models/bert_text_Adam_lr0.0001_bz256.76.hdf5
Epoch 77/250
4903/4903 [==============================] - 0s 30us/step - loss: 0.5425 - acc: 0.8313 - val_loss: 1.0645 - val_acc: 0.6639

Epoch 00077: saving model to models/bert_text_Adam_lr0.0001_bz256.77.hdf5
Epoch 78/250
4903/4903 [==============================] - 0s 26us/step - loss: 0.5445 - acc: 0.8436 - val_loss: 1.0678 - val_acc: 0.6697

Epoch 00078: saving model to models/bert_text_Adam_lr0.0001_bz256.78.hdf5
Epoch 79/250
4903/4903 [==============================] - 0s 30us/step - loss

4903/4903 [==============================] - 0s 28us/step - loss: 0.3990 - acc: 0.8886 - val_loss: 1.1031 - val_acc: 0.6639

Epoch 00113: saving model to models/bert_text_Adam_lr0.0001_bz256.113.hdf5
Epoch 114/250
4903/4903 [==============================] - 0s 27us/step - loss: 0.3921 - acc: 0.8888 - val_loss: 1.1059 - val_acc: 0.6639

Epoch 00114: saving model to models/bert_text_Adam_lr0.0001_bz256.114.hdf5
Epoch 115/250
4903/4903 [==============================] - 0s 36us/step - loss: 0.3969 - acc: 0.8866 - val_loss: 1.1056 - val_acc: 0.6574

Epoch 00115: saving model to models/bert_text_Adam_lr0.0001_bz256.115.hdf5
Epoch 116/250
4903/4903 [==============================] - 0s 27us/step - loss: 0.3893 - acc: 0.8909 - val_loss: 1.1060 - val_acc: 0.6639

Epoch 00116: saving model to models/bert_text_Adam_lr0.0001_bz256.116.hdf5
Epoch 117/250
4903/4903 [==============================] - 0s 25us/step - loss: 0.3957 - acc: 0.8850 - val_loss: 1.1107 - val_acc: 0.6623

Epoch 00117: saving

4903/4903 [==============================] - 0s 28us/step - loss: 0.3003 - acc: 0.9164 - val_loss: 1.1687 - val_acc: 0.6591

Epoch 00151: saving model to models/bert_text_Adam_lr0.0001_bz256.151.hdf5
Epoch 152/250
4903/4903 [==============================] - 0s 29us/step - loss: 0.3002 - acc: 0.9135 - val_loss: 1.1623 - val_acc: 0.6648

Epoch 00152: saving model to models/bert_text_Adam_lr0.0001_bz256.152.hdf5
Epoch 153/250
4903/4903 [==============================] - 0s 24us/step - loss: 0.3064 - acc: 0.9086 - val_loss: 1.1582 - val_acc: 0.6599

Epoch 00153: saving model to models/bert_text_Adam_lr0.0001_bz256.153.hdf5
Epoch 154/250
4903/4903 [==============================] - 0s 26us/step - loss: 0.2887 - acc: 0.9237 - val_loss: 1.1694 - val_acc: 0.6639

Epoch 00154: saving model to models/bert_text_Adam_lr0.0001_bz256.154.hdf5
Epoch 155/250
4903/4903 [==============================] - 0s 32us/step - loss: 0.2866 - acc: 0.9168 - val_loss: 1.1670 - val_acc: 0.6639

Epoch 00155: saving

4903/4903 [==============================] - 0s 34us/step - loss: 0.2362 - acc: 0.9384 - val_loss: 1.2307 - val_acc: 0.6631

Epoch 00189: saving model to models/bert_text_Adam_lr0.0001_bz256.189.hdf5
Epoch 190/250
4903/4903 [==============================] - 0s 32us/step - loss: 0.2349 - acc: 0.9353 - val_loss: 1.2261 - val_acc: 0.6607

Epoch 00190: saving model to models/bert_text_Adam_lr0.0001_bz256.190.hdf5
Epoch 191/250
4903/4903 [==============================] - 0s 24us/step - loss: 0.2294 - acc: 0.9380 - val_loss: 1.2272 - val_acc: 0.6615

Epoch 00191: saving model to models/bert_text_Adam_lr0.0001_bz256.191.hdf5
Epoch 192/250
4903/4903 [==============================] - 0s 32us/step - loss: 0.2313 - acc: 0.9366 - val_loss: 1.2220 - val_acc: 0.6631

Epoch 00192: saving model to models/bert_text_Adam_lr0.0001_bz256.192.hdf5
Epoch 193/250
4903/4903 [==============================] - 0s 35us/step - loss: 0.2264 - acc: 0.9406 - val_loss: 1.2423 - val_acc: 0.6623

Epoch 00193: saving

4903/4903 [==============================] - 0s 27us/step - loss: 0.1805 - acc: 0.9513 - val_loss: 1.2970 - val_acc: 0.6582

Epoch 00227: saving model to models/bert_text_Adam_lr0.0001_bz256.227.hdf5
Epoch 228/250
4903/4903 [==============================] - 0s 27us/step - loss: 0.1839 - acc: 0.9498 - val_loss: 1.3013 - val_acc: 0.6648

Epoch 00228: saving model to models/bert_text_Adam_lr0.0001_bz256.228.hdf5
Epoch 229/250
4903/4903 [==============================] - 0s 26us/step - loss: 0.1827 - acc: 0.9494 - val_loss: 1.3032 - val_acc: 0.6623

Epoch 00229: saving model to models/bert_text_Adam_lr0.0001_bz256.229.hdf5
Epoch 230/250
4903/4903 [==============================] - 0s 27us/step - loss: 0.1808 - acc: 0.9547 - val_loss: 1.3132 - val_acc: 0.6631

Epoch 00230: saving model to models/bert_text_Adam_lr0.0001_bz256.230.hdf5
Epoch 231/250
4903/4903 [==============================] - 0s 29us/step - loss: 0.1894 - acc: 0.9490 - val_loss: 1.2955 - val_acc: 0.6631

Epoch 00231: saving

In [31]:
print('max_acc_arg', max_acc_arg)
print('min_loss_arg', min_loss_arg)

78
60


In [32]:
epoch = input("\n\nWhich epoch to load?\nAns: ")
epoch = int(epoch)
load_model_path = '%s.%02d.hdf5' % (model_path, epoch)
print('Loading model - ', load_model_path)
model = load_model(load_model_path, custom_objects={'loss': 'categorical_crossentropy'})
# print(model.summary())



Which epoch to load?
Ans: 78
Loading model -  models/bert_text_Adam_lr0.0001_bz256.78.hdf5


In [33]:
predictions = model.predict(dev_articles_emb)
predictions = predictions.argmax(axis=1)

# writing predictions to file
task_TC_output_file = "BERT_model-output-TC.txt"
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

with open(task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(dev_article_ids, predictions, dev_span_starts,
                                                            dev_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, index2label[prediction], span_start, span_end))
print("Predictions written to file " + task_TC_output_file)

Predictions written to file BERT_model-output-TC.txt


In [34]:
predictions = model.predict(test_articles_emb)
predictions = predictions.argmax(axis=1)

# writing predictions to file
test_task_TC_output_file = "test-output-TC.txt"
test_article_ids, test_span_starts, test_span_ends, test_labels = read_predictions_from_file(test_template_labels_file)

with open(test_task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(test_article_ids, predictions, test_span_starts,
                                                            test_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, index2label[prediction], span_start, span_end))
print("Predictions written to file " + test_task_TC_output_file)

Predictions written to file test-output-TC.txt
